In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from fancyimpute import KNN

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [ ]:
df_train = pd.read_csv("public-train.csv", sep="|")
df_test = pd.read_csv("public-test.csv", sep="|")
df = pd.concat([df_train, df_test]).drop(['author_id', 'publisher_id'], axis=1)



In [ ]:
df['book_age'] = 2021 - pd.to_numeric(df['published_date'].str[-4:], errors='coerce')
df['volume'] = df['dimension_0'] * df['dimension_1'] * df['dimension_2']
df.drop(['published_date', 'dimension_0', 'dimension_1', 'dimension_2'], axis=1, inplace=True)

df['bookformat'] = df['bookformat'].str.replace("paperback", "Paperback").str.replace(
    "(Unknown Binding)|(Spiral-bound)|(Library Binding)|(Trade Paperback)|(Comics)|(Board Book)|(Nook)", "Others")
df[['Kindle', "MMPaperback", "Others", "Paperback", "Ebook"]] = pd.get_dummies(df['bookformat'], drop_first=True)

In [ ]:
df_train.describe()

,pages,weight,rating_value_0,rating_value_1,rating_count_0,rating_count_1,dimension_0,dimension_1,dimension_2,genre_0_weight,genre_1_weight,genre_2_weight,genre_3_weight,genre_4_weight,genre_5_weight,genre_6_weight,genre_7_weight,genre_8_weight,genre_9_weight,price
count,3425.000000,512.000000,3487.000000,548.000000,3.497000e+03,3497.000000,506.000000,506.000000,495.000000,3394.000000,3351.000000,3322.000000,3302.000000,3270.000000,3240.00000,3212.000000,3172.000000,3136.000000,3100.000000,523.000000
mean,332.585985,2370.206328,4.082753,4.515146,5.587966e+04,376.358593,15.449111,3.629071,22.092970,0.426730,0.198165,0.119124,0.077935,0.054976,0.04166,0.032699,0.026434,0.021875,0.018174,466.117628
std,147.576876,2224.992798,0.293261,0.281626,1.849122e+05,2061.585820,3.456639,3.849360,3.249474,0.164264,0.069955,0.046482,0.031668,0.022640,0.01698,0.013457,0.011010,0.009474,0.008107,134.962630
min,20.000000,400.070000,3.030000,3.200000,1.000000e+00,1.000000,1.470000,0.230000,1.020000,0.020000,0.010000,0.010000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,119.553335
25%,256.000000,612.350000,3.890000,4.400000,3.867000e+03,1.000000,13.970000,2.130000,20.950000,0.310000,0.150000,0.090000,0.060000,0.040000,0.03000,0.020000,0.020000,0.020000,0.010000,360.262404
50%,332.000000,839.150000,4.090000,4.600000,1.345700e+04,1.000000,15.240000,2.790000,22.860000,0.390000,0.200000,0.120000,0.080000,0.050000,0.04000,0.030000,0.030000,0.020000,0.020000,464.578303
75%,400.000000,4014.290000,4.260000,4.700000,4.404100e+04,1.000000,16.360000,3.580000,24.130000,0.500000,0.240000,0.150000,0.100000,0.070000,0.05000,0.040000,0.030000,0.030000,0.020000,536.673085
max,1248.000000,7212.110000,5.000000,5.000000,3.803071e+06,40409.000000,30.480000,27.430000,28.910000,1.000000,0.500000,0.330000,0.250000,0.150000,0.12000,0.110000,0.070000,0.070000,0.060000,989.138514


In [ ]:
df

,description,bookformat,bookedition,pages,reading_age,lexile_measure,grade_level,weight,rating_value_0,rating_value_1,rating_count_0,rating_count_1,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_0_weight,genre_1_weight,genre_2_weight,genre_3_weight,genre_4_weight,genre_5_weight,genre_6_weight,genre_7_weight,genre_8_weight,genre_9_weight,price,book_age,volume,Kindle,MMPaperback,Others,Paperback,Ebook,count_per_year
0,"Just after the Second World War, in the small ...",Hardcover,NaN,309.0,NaN,NaN,NaN,NaN,3.76,NaN,26625,1,Historical Fiction,Fiction,Historical,Audiobook,Romance,Books About Books,Adult,Adult Fiction,British Literature,Chick Lit,0.45,0.22,0.08,0.06,0.05,0.04,0.03,0.02,0.02,0.02,NaN,NaN,NaN,0,0,0,0,0,NaN
1,"Blame it on Hawaii’s rainbows, sparkling beach...",Paperback,NaN,NaN,NaN,NaN,NaN,NaN,4.48,NaN,21,1,Memoir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,NaN
2,"The Pulitzer Prize–winning, bestselling author...",Hardcover,NaN,496.0,NaN,NaN,NaN,NaN,4.56,NaN,59885,1,Nonfiction,History,Politics,Race,Social Justice,Audiobook,Sociology,Anti Racist,American History,African American,0.42,0.22,0.08,0.08,0.05,0.05,0.04,0.02,0.02,0.02,NaN,NaN,NaN,0,0,0,0,0,NaN
3,THINGS ARE ABOUT TO GET SERIOUS FOR HARRY DRES...,Hardcover,First Edition,418.0,NaN,NaN,NaN,NaN,4.39,NaN,26643,1,Fantasy,Urban Fantasy,Fiction,Magic,Paranormal,Audiobook,Vampires,Mystery,Supernatural,Fae,0.41,0.30,0.08,0.05,0.04,0.04,0.03,0.02,0.02,0.02,NaN,NaN,NaN,0,0,0,0,0,NaN
4,The Romanovs were the most successful dynasty ...,Hardcover,NaN,784.0,NaN,NaN,NaN,NaN,3.93,NaN,11772,1,History,Nonfiction,Russia,Biography,Historical,Russian History,Audiobook,Politics,European History,Romanovs,0.42,0.30,0.09,0.08,0.04,0.02,0.01,0.01,0.01,0.01,NaN,NaN,NaN,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,World War I stands as one of history's most se...,Hardcover,NaN,448.0,NaN,NaN,NaN,771.11,4.15,4.6,7953,409,History,Nonfiction,World War I,War,Military History,Politics,Military Fiction,European History,World History,Historical,0.50,0.22,0.09,0.07,0.02,0.02,0.02,0.02,0.02,0.01,352263.0,10.0,1660.651000,0,0,0,0,0,723.000000
496,"A sweeping, evocative epic of two women's inte...",Hardcover,NaN,589.0,NaN,NaN,NaN,748.43,3.65,3.8,39510,2920,Historical Fiction,Fiction,China,Historical,Asia,Asian Literature,Adult,Adult Fiction,Audiobook,Novels,0.36,0.33,0.10,0.06,0.04,0.02,0.02,0.02,0.02,0.02,307364.0,8.0,1299.481272,0,0,0,0,0,4390.000000
497,She could remember standing in a park near the...,Hardcover,NaN,38.0,6 - 9 years,1060L,1 - 4,7121.39,3.90,4.6,4330,103,Picture Books,Biography,Nonfiction,Childrens,History,Adventure,Biography Memoir,Historical,Womens,Juvenile,0.35,0.21,0.14,0.09,0.08,0.05,0.03,0.03,0.02,0.02,274159.0,10.0,694.450224,0,0,0,0,0,393.636364
498,"Following on the success of Tender and Ripe, t...",Hardcover,NaN,544.0,NaN,NaN,NaN,1564.89,4.31,4.5,799,36,Cookbooks,Cooking,Nonfiction,Food,Reference,Food Writing,Essays,Memoir,Foodie,Culinary,0.39,0.19,0.13,0.12,0.05,0.03,0.03,0.03,0.03,0.03,489992.0,8.0,2108.850240,0,0,0,0,0,88.777778


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3497 entries, 0 to 3549
Data columns (total 39 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   author_id       3487 non-null   object 
 1   description     3468 non-null   object 
 2   bookformat      3484 non-null   object 
 3   bookedition     228 non-null    object 
 4   pages           3425 non-null   float64
 5   published_date  551 non-null    object 
 6   publisher_id    551 non-null    object 
 7   reading_age     126 non-null    object 
 8   lexile_measure  88 non-null     object 
 9   grade_level     100 non-null    object 
 10  weight          512 non-null    float64
 11  rating_value_0  3487 non-null   float64
 12  rating_value_1  548 non-null    float64
 13  rating_count_0  3497 non-null   int64  
 14  rating_count_1  3497 non-null   int64  
 15  dimension_0     506 non-null    float64
 16  dimension_1     506 non-null    float64
 17  dimension_2     495 non-null    f

In [ ]:
age = pd.DataFrame()
age[['lower_age', 'upper_age']] = df['reading_age'].str.replace("years|and|\-", "").str.replace("up","21").str.replace("Baby", "4").str.split(expand=True)

df['reading_age'] = (age['lower_age'].astype(float)+age['upper_age'].astype(float))/2

grade = pd.DataFrame()
grade[['lower_grade', 'upper_grade']] = df['grade_level'].str.replace("and|up|\-", "").str.replace("Preschool|Kindergarten", "0").str.split(expand=True)

df['grade_level'] = (grade['lower_grade'].astype(float)+grade['upper_grade'].astype(float))/2

## DROP

In [ ]:
genres = ['genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4',
          'genre_5', 'genre_6', 'genre_7', 'genre_8', 'genre_9']
weights = ['genre_0_weight', 'genre_1_weight', 'genre_2_weight', 'genre_3_weight',
           'genre_4_weight', 'genre_5_weight', 'genre_6_weight', 'genre_7_weight',
           'genre_8_weight', 'genre_9_weight']
cols = ['pages', 'weight', 
             'volume', 'book_age', 'price', 'rating_value_0', 'count_per_year'] + genres + weights

df_drop = df[cols].dropna(subset=['price']).copy()

for i in range(10):
  df_drop[genres[i]] = df_drop.groupby(genres[i])['price'].transform('mean')
  df_drop[genres[i]] = df_drop[genres[i]]*df_drop[weights[i]]

df_drop['genres'] = df_drop['genre_9']
for i in range(9):
  df_drop['genres'] = df_drop['genres'] + df_drop[genres[i]]

df_drop.drop(df_drop[df_drop['price'] == 0].index, inplace=True)
df_drop.drop((df_drop[df_drop['rating_value_0'] == 0]).index, inplace=True)
df_drop.drop(df_drop[df_train['rating_value_0'] < 3].index, inplace=True)
df_drop['price'] = np.sqrt(df_drop['price'])
df_drop['pages'] = np.sqrt(df_drop['pages'])

df_drop.drop(genres + weights, axis=1, inplace=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [ ]:
knn_imputer = KNN()

df_drop.iloc[:, :] = knn_imputer.fit_transform(df_drop)

Imputing row 1/1002 with 0 missing, elapsed time: 0.179
Imputing row 101/1002 with 0 missing, elapsed time: 0.183
Imputing row 201/1002 with 6 missing, elapsed time: 0.185
Imputing row 301/1002 with 0 missing, elapsed time: 0.187
Imputing row 401/1002 with 3 missing, elapsed time: 0.189
Imputing row 501/1002 with 0 missing, elapsed time: 0.191
Imputing row 601/1002 with 0 missing, elapsed time: 0.193
Imputing row 701/1002 with 0 missing, elapsed time: 0.195
Imputing row 801/1002 with 0 missing, elapsed time: 0.197
Imputing row 901/1002 with 0 missing, elapsed time: 0.199
Imputing row 1001/1002 with 0 missing, elapsed time: 0.201


In [ ]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
X_drop = df_drop.drop(['price'], axis=1).copy()
y_drop = df_drop['price'].copy()

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
model = RandomForestRegressor()
scores = cross_val_score(model, X_drop, y_drop, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)
print('RMSE: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

RMSE: -96.554 (13.614)


## DF full

In [ ]:
genres = ['genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4',
          'genre_5', 'genre_6', 'genre_7', 'genre_8', 'genre_9']
weights = ['genre_0_weight', 'genre_1_weight', 'genre_2_weight', 'genre_3_weight',
           'genre_4_weight', 'genre_5_weight', 'genre_6_weight', 'genre_7_weight',
           'genre_8_weight', 'genre_9_weight']
cols = ['pages', 'weight', 
             'volume', 'book_age', 'price', 'rating_value_0', 'rating_count_0'] + genres + weights
df_train = df[cols][:3550].copy()
df_test = df[cols][3550:].copy()

for i in range(10):
  df_train[genres[i]] = df_train[genres[i]].map(df_train[genres[i]].value_counts())

  df_test[genres[i]] = df_test[genres[i]].map(df_test[genres[i]].value_counts())


df_train['price'] = np.sqrt(df_train['price'])
df_train['pages'] = np.sqrt(df_train['pages'])



In [ ]:
X_train = df_train.drop('price', axis=1).copy()
y_train = df_train['price'].copy()

cond = np.isnan(df_train['price'])
X_train_train = X_train[np.logical_not(cond)].copy()
y_train_train = y_train[np.logical_not(cond)].copy()

X_train_test = df_train.drop('price', axis=1)[cond]

In [ ]:
X_train_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 543 entries, 9 to 3541
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pages           511 non-null    float64
 1   weight          457 non-null    float64
 2   volume          443 non-null    float64
 3   book_age        498 non-null    float64
 4   rating_value_0  543 non-null    float64
 5   rating_count_0  543 non-null    int64  
 6   genre_0         492 non-null    float64
 7   genre_1         480 non-null    float64
 8   genre_2         475 non-null    float64
 9   genre_3         471 non-null    float64
 10  genre_4         460 non-null    float64
 11  genre_5         453 non-null    float64
 12  genre_6         445 non-null    float64
 13  genre_7         442 non-null    float64
 14  genre_8         435 non-null    float64
 15  genre_9         430 non-null    float64
 16  genre_0_weight  492 non-null    float64
 17  genre_1_weight  480 non-null    fl

In [ ]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor


cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
model = XGBRegressor()
scores = cross_val_score(model, X_train_train, y_train_train, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)
print('RMSE: %.3f (%.3f)\nPercentage: %.3f' % (-np.mean(scores), np.std(scores), 
                                               -np.mean(scores)/(y_train_train.max()-y_train_train.min())))

RMSE: 119.026 (9.173)
Percentage: 0.120


In [ ]:
model.fit(X_train_train, y_train_train)
y_train_pred = model.predict(X_train_test)


[02:50:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
X_train_train['price'] = y_train_train
X_train_test['price'] = y_train_pred

df_train_final = pd.concat([X_train_train, X_train_test])

In [ ]:
df_train.head()

,pages,weight,volume,book_age,price,rating_value_0,rating_count_0,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_0_weight,genre_1_weight,genre_2_weight,genre_3_weight,genre_4_weight,genre_5_weight,genre_6_weight,genre_7_weight,genre_8_weight,genre_9_weight
0,17.578396,NaN,NaN,NaN,NaN,3.76,26625,170.0,495.0,94.0,260.0,71.0,9.0,254.0,97.0,14.0,32.0,0.45,0.22,0.08,0.06,0.05,0.04,0.03,0.02,0.02,0.02
1,NaN,NaN,NaN,NaN,NaN,4.48,21,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22.271057,NaN,NaN,NaN,NaN,4.56,59885,564.0,72.0,34.0,23.0,8.0,341.0,13.0,4.0,17.0,28.0,0.42,0.22,0.08,0.08,0.05,0.05,0.04,0.02,0.02,0.02
3,20.445048,NaN,NaN,NaN,NaN,4.39,26643,408.0,24.0,347.0,31.0,48.0,341.0,5.0,22.0,43.0,4.0,0.41,0.30,0.08,0.05,0.04,0.04,0.03,0.02,0.02,0.02
4,28.000000,NaN,NaN,NaN,NaN,3.93,11772,79.0,341.0,3.0,120.0,66.0,1.0,325.0,17.0,2.0,1.0,0.42,0.30,0.09,0.08,0.04,0.02,0.01,0.01,0.01,0.01


In [ ]:
df_train_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3550 entries, 9 to 3549
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pages           3451 non-null   float64
 1   weight          519 non-null    float64
 2   volume          499 non-null    float64
 3   book_age        566 non-null    float64
 4   rating_value_0  3540 non-null   float64
 5   rating_count_0  3550 non-null   int64  
 6   genre_0         3400 non-null   float64
 7   genre_1         3353 non-null   float64
 8   genre_2         3322 non-null   float64
 9   genre_3         3302 non-null   float64
 10  genre_4         3270 non-null   float64
 11  genre_5         3240 non-null   float64
 12  genre_6         3212 non-null   float64
 13  genre_7         3172 non-null   float64
 14  genre_8         3136 non-null   float64
 15  genre_9         3100 non-null   float64
 16  genre_0_weight  3400 non-null   float64
 17  genre_1_weight  3353 non-null   f

In [ ]:
X_final = df_train_final.drop('price', axis=1).copy()

y_final = df_train_final['price'].copy()

In [ ]:
X_test = df_test.drop('price', axis=1).copy()
y_test = df_test['price'].copy()


In [ ]:
model = XGBRegressor()
model.fit(X_final, y_final)
y_pred = model.predict(X_test)
model.score(X_test, np.sqrt(y_test))

[02:50:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.23836125553040466

In [ ]:
from sklearn.metrics import mean_squared_error
y_sqr =  np.square(y_pred)
np.sqrt(mean_squared_error(y_test, y_sqr))

176580.36367793096

In [ ]:
y_test.mean()

253518.7

In [ ]:
np.sqrt(((np.square(y_test - y_sqr)).sum())/(len(y_test)-1))

175594.72075298283

In [ ]:
y_train.isna().sum()

0

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def nlp_preprocessing(total_text):
    if type(total_text) is not int:
        string = ""
        for word in total_text.split():
        # if the word is a not a stop word then retain that word from the data
            if not word in stop_words:
                string += word + " "
        
    return string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import re
import string
df_train = df.iloc[:3550].dropna(subset=['description']).copy()
df_train['description'] = df_train['description'].apply(lambda x: nlp_preprocessing(x))
df_train['description'] = df_train['description'].str.lower()
df_train['description'] = df_train['description'].astype(str).apply(lambda x : re.sub("[^A-Za-z]"," ",x))
df_train['description'] = df_train['description'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import KNNImputer

genres = ['genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4',
          'genre_5', 'genre_6', 'genre_7', 'genre_8', 'genre_9']
weights = ['genre_0_weight', 'genre_1_weight', 'genre_2_weight', 'genre_3_weight',
           'genre_4_weight', 'genre_5_weight', 'genre_6_weight', 'genre_7_weight',
           'genre_8_weight', 'genre_9_weight']
cols = ['pages', 'weight', 
             'volume', 'book_age', 'rating_value_0', 'rating_count_0'] + genres + weights
df_train = df[cols + ['price']][:3550].copy()
df_test = df[cols + ['price']][3550:].copy()

for i in range(10):
  df_train[genres[i]] = df_train[genres[i]].map(df_train[genres[i]].value_counts())

  df_test[genres[i]] = df_test[genres[i]].map(df_test[genres[i]].value_counts())


df_train['price'] = np.sqrt(df_train['price'])
df_train['pages'] = np.sqrt(df_train['pages'])

get_numeric_data=FunctionTransformer(lambda x:x[cols],
                                       validate=False)


numeric_pipeline = Pipeline([
    ('selector',get_numeric_data),
])



import xgboost as xgb

from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV
params = {
    'clf__n_estimators' : list(np.arange(100,300,50)),
    'clf__max_depth' : [3,4,5,6,7],
    'clf__learning_rate': [0.01,0.1,0.01],
    'clf__reg_lambda': list(np.arange(0.1,0.9,0.1)),
    'clf__colsample_bytree' : list(np.arange(0.1,0.8,0.1)),
    'clf__importance_type': ['gain','weight','cover','total_gain','total_cover'],
    'clf__booster': ['gbtree','gblinear','dart']
}

pl = Pipeline([
    ('union',FeatureUnion([
        ('numeric',numeric_pipeline)
    ])),
    ('clf',xgb.XGBRegressor(verbosity=0,objective='reg:squarederror'))
])

from sklearn.metrics import mean_squared_error, make_scorer
def my_custom_loss_func(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_pred,y_true))
  
score = make_scorer(my_custom_loss_func, greater_is_better=False)

grid = GridSearchCV(pl,param_grid=params,cv=5,scoring=score)

In [ ]:
X_train = df_train.drop('price', axis=1).copy()
y_train = df_train['price'].copy()

cond = np.isnan(df_train['price'])
X_train_train = X_train[np.logical_not(cond)].copy()
y_train_train = y_train[np.logical_not(cond)].copy()


X_train_test = df_train.drop('price', axis=1)[cond]
grid.fit(X_train_train, y_train_train)

In [ ]:
grid.best_score_